# FixedIncomeLib Product Tests

This notebook exercises all of the products defined in `product.linear_products` and prints their attributes via the display visitors.

---

## 1. Imports and Setup

We import the necessary classes and visitors.

---

In [1]:
from fixedincomelib.product.linear_products import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    InterestRateStream,
)
from fixedincomelib.product.product_display_visitor import (
    CashflowVisitor,
    IborCashflowVisitor,
    OvernightCashflowVisitor,
    FutureVisitor,
    RfrFutureVisitor,
    IborSwapVisitor,
    OvernightSwapVisitor,
)

# Set up some common parameters
notional = 10_000
LONG = "LONG"
SHORT = "SHORT"

print("Setup complete.")

Setup complete.


## 2. Bullet Cashflow

Create a `ProductBulletCashflow` and display its attributes.

---

In [2]:
bullet = ProductBulletCashflow("2025-12-31", "USD", notional, LONG)
print("ProductBulletCashflow →")
print(bullet.accept(CashflowVisitor()), "\n")


ProductBulletCashflow →
         Attribute       Value
0  TerminationDate  2025-12-31
1         Currency         USD
2         Notional       10000
3      LongOrShort        LONG 



## 3. IBOR Cashflow

Create a `ProductIborCashflow` and display its attributes.

---

In [3]:
# ProductIborCashflow
ibor_cf = ProductIborCashflow(
    startDate="2025-06-01",
    endDate="2025-09-01",
    index="USD-LIBOR-BBA-3M",
    spread=0.001,
    notional=notional,
    longOrShort=SHORT,
)

print("ProductIborCashflow →")
print(ibor_cf.accept(IborCashflowVisitor()))


ProductIborCashflow →
       Attribute             Value
0   AccrualStart        2025-06-01
1     AccrualEnd        2025-09-01
2  AccrualFactor              0.25
3          Index  USD-LIBOR-BBA-3M
4         Spread             0.001
5       Notional             10000
6       Currency               USD
7    LongOrShort             SHORT


## 4. Overnight Index Cashflow

Create a `ProductOvernightIndexCashflow` and display its attributes.

---

In [4]:
# ProductOvernightCashflow via tenor
ois_cf_tenor = ProductOvernightIndexCashflow(
    effectiveDate='2025-06-01',
    termOrEnd='3M',
    index='SOFR-1B',
    compounding='COMPOUND',
    spread=0.0005,
    notional=2e5,
    longOrShort='Long'
)
print("ProductOvernightCashflow (tenor) →")
print(ois_cf_tenor.accept(OvernightCashflowVisitor()), "\n")


ProductOvernightCashflow (tenor) →
         Attribute       Value
0    EffectiveDate  2025-06-01
1  TerminationDate  2025-09-02
2            Index     SOFR-1B
3      Compounding    COMPOUND
4           Spread      0.0005
5         Notional    200000.0
6         Currency         USD
7      LongOrShort        Long 



In [5]:
# ProductOvernightCashflow via explicit date
ois_cf_date = ProductOvernightIndexCashflow(
    effectiveDate='2025-06-01',
    termOrEnd='2025-12-01',
    index='SOFR-1B',
    compounding='COMPOUND',
    spread=0.0005,
    notional=2e5,
    longOrShort='Long'
)
print("ProductOvernightCashflow (date) →")
print(ois_cf_date.accept(OvernightCashflowVisitor()), "\n")


ProductOvernightCashflow (date) →
         Attribute       Value
0    EffectiveDate  2025-06-01
1  TerminationDate  2025-12-01
2            Index     SOFR-1B
3      Compounding    COMPOUND
4           Spread      0.0005
5         Notional    200000.0
6         Currency         USD
7      LongOrShort        Long 



## 5. Future

Create a `ProductFuture` and display its attributes.

---

In [6]:
# ProductFuture (IBOR-based)
future = ProductFuture(
    effectiveDate="2025-06-01",
    index="EURIBOR-3M",
    strike=99.5,
    notional=notional,
    longOrShort=LONG,
)
print("ProductFuture (IBOR) →")
print(future.accept(FutureVisitor()))


ProductFuture (IBOR) →
        Attribute       Value
0  ExpirationDate  2025-05-29
1   EffectiveDate  2025-06-01
2    MaturityDate  2025-09-01
3   AccrualFactor        0.25
4        Currency         EUR
5        Notional       10000
6          Strike        99.5
7     LongOrShort        LONG


## 6. RFR Future

Create a `ProductRfrFuture` and display its attributes.

---


In [7]:
# ProductRfrFuture (OIS-based)
rfr_future = ProductRfrFuture(
    effectiveDate="2025-06-01",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="AVERAGE",
    strike=99.7,
    notional=notional,
    longOrShort=SHORT,
)
print("ProductRfrFuture (OIS) →")
print(rfr_future.accept(RfrFutureVisitor()), "\n")


ProductRfrFuture (OIS) →
         Attribute       Value
0     MaturityDate  2025-09-02
1    EffectiveDate  2025-06-01
2  TerminationDate  2025-09-02
3    AccrualFactor    0.252778
4      Compounding     AVERAGE
5            Index     SOFR-1B
6           Strike        99.7
7         Notional       10000
8         Currency         USD
9      LongOrShort       SHORT 



## 7. Interest Rate Streams

### 7.1 IBOR Stream

Build a 3M‐frequency IBOR stream and display each leg.

---

In [8]:
ibor_stream = InterestRateStream(
    startDate="2025-01-01",
    endDate="2025-12-31",
    frequency="3M",
    iborIndex="USD-LIBOR-BBA-3M",
    overnightIndex=None,
    fixedRate=None,
    notional=notional,
    position=LONG,
)
for i in range(ibor_stream.numProducts):
    cf = ibor_stream.element(i)
    vis = IborCashflowVisitor() if cf.prodType == "ProductIborCashflow" else CashflowVisitor()
    print(f"\n-- leg #{i}")
    print(cf.accept(vis))



-- leg #0
       Attribute             Value
0   AccrualStart        2025-01-02
1     AccrualEnd        2025-03-31
2  AccrualFactor          0.247222
3          Index  USD-LIBOR-BBA-3M
4         Spread               0.0
5       Notional             10000
6       Currency               USD
7    LongOrShort              LONG

-- leg #1
       Attribute             Value
0   AccrualStart        2025-03-31
1     AccrualEnd        2025-06-30
2  AccrualFactor              0.25
3          Index  USD-LIBOR-BBA-3M
4         Spread               0.0
5       Notional             10000
6       Currency               USD
7    LongOrShort              LONG

-- leg #2
       Attribute             Value
0   AccrualStart        2025-06-30
1     AccrualEnd        2025-09-30
2  AccrualFactor              0.25
3          Index  USD-LIBOR-BBA-3M
4         Spread               0.0
5       Notional             10000
6       Currency               USD
7    LongOrShort              LONG

-- leg #3
       Attr

### 7.2 OIS Stream

Build a 3M‐frequency OIS stream and display each leg.

---

In [9]:
ois_stream = InterestRateStream(
    startDate="2025-01-01",
    endDate="2025-12-31",
    frequency="3M",
    iborIndex=None,
    overnightIndex="SOFR-1B",
    fixedRate=None,
    notional=notional,
    position=SHORT,
)
for i in range(ois_stream.numProducts):
    cf = ois_stream.element(i)
    print(f"\n-- leg #{i}")
    print(cf.accept(OvernightCashflowVisitor()))



-- leg #0
         Attribute       Value
0    EffectiveDate  2025-01-02
1  TerminationDate  2025-03-31
2            Index     SOFR-1B
3      Compounding    COMPOUND
4           Spread         0.0
5         Notional       10000
6         Currency         USD
7      LongOrShort       SHORT

-- leg #1
         Attribute       Value
0    EffectiveDate  2025-03-31
1  TerminationDate  2025-06-30
2            Index     SOFR-1B
3      Compounding    COMPOUND
4           Spread         0.0
5         Notional       10000
6         Currency         USD
7      LongOrShort       SHORT

-- leg #2
         Attribute       Value
0    EffectiveDate  2025-06-30
1  TerminationDate  2025-09-30
2            Index     SOFR-1B
3      Compounding    COMPOUND
4           Spread         0.0
5         Notional       10000
6         Currency         USD
7      LongOrShort       SHORT

-- leg #3
         Attribute       Value
0    EffectiveDate  2025-09-30
1  TerminationDate  2025-12-31
2            Index     SOF

## 8. Swaps

### 8.1 IBOR Swap

Create a `ProductIborSwap` and display its summary via the visitor.

---

In [10]:
ibor_swap = ProductIborSwap(
    effectiveDate="2025-01-01",
    maturityDate="2026-01-01",
    frequency="6M",
    iborIndex="USD-LIBOR-BBA-3M",
    spread=0.0,
    fixedRate=0.015,
    notional=notional,
    position=SHORT,
)
print(ibor_swap.accept(IborSwapVisitor()))

       Attribute             Value
0  EffectiveDate        2025-01-01
1   MaturityDate        2026-01-01
2      FixedRate             0.015
3          Index  USD-LIBOR-BBA-3M
4       PayFixed              True
5       Notional             10000
6       Currency               USD
7    LongOrShort             SHORT


### 8.2 OIS Swap

Create a `ProductOvernightSwap` and display its summary via the visitor.

---

In [11]:
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-01-01",
    maturityDate="2026-01-01",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.015,
    notional=notional,
    position=LONG,
)
print(ois_swap.accept(OvernightSwapVisitor()))

       Attribute       Value
0  EffectiveDate  2025-01-01
1   MaturityDate  2026-01-01
2      FixedRate       0.015
3          Index     SOFR-1B
4       PayFixed       False
5       Notional       10000
6       Currency         USD
7    LongOrShort        LONG
